## Day 3 - Conversation AI - aka Chatbot

In [1]:
import os
import ollama
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

/opt/anaconda3/envs/udemy_llm/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()

True

In [3]:
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', '')


In [4]:
openai = OpenAI()
gpt_model = 'gpt-4o-mini'
#openai initialize

In [5]:
client = ollama.Client(host=os.getenv('OLLAMA_HOST', 'http://localhost:11434'))
#ollama initialize

In [6]:
system_message = "You are helpful assistant"


### Reminder of the structure of prompt messages to OpenAI

[</br>
    {'role' : 'system', 'content': 'system messages here'},</br>
    {'role' : 'user', 'content' : first user prompt here'},</br>
    {'role' : 'assistant', 'content' : 'the assistant response'},</br>
    {'role' : 'user', 'content' : 'the new user prompt'}</br>
]</br>

we will write a function chat(message, history)</br>
where: message is the prompt to use history is a list of pairs of user message with assistant</br>
[</br>
    ['user said this', 'assistant replied']</br>
    ['then user said this', 'assistance replied again']</br>

]</br>
</br>
we will convert this history into the prompt style for OpenAI, then call OpenAI


In [7]:
system_message = "You are helpful assistant in a clothes store. you should try to gently encourage \
    the customer to try items that are on sale. Hats are 60% off, and most other items are 50% off.\
    For example, if the customer says 'I'm looking to buy a hat',\
    you could reply something like, 'wonderful - we have lots of hats - incruding several that are part of our sales event.'\
    Encourage the customer to buy hats if they are unsure what to get." 

In [8]:
def gpt_chat(message, history):
    messages = [{'role': "system", 'content' : system_message}]
    for user_message, assitant_message in history:
        messages.append({'role': 'user', 'content': user_message})
        messages.append({'role': 'assistant', 'content': assitant_message})
    
    messages.append({'role': 'user', 'content': message})

    print('History is')
    print(history)
    print('and messages is')
    print(messages)

    stream = openai.chat.completions.create(
        model = gpt_model,
        messages = messages,
        stream = True
    )
    response = ''
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

    

In [9]:
def llama_chat(message, history):
    messages = [{'role': "system", 'content' : system_message}]
    for user_message, assitant_message in history:
        messages.append({'role': 'user', 'content': user_message})
        messages.append({'role': 'assistant', 'content': assitant_message})
    
    messages.append({'role': 'user', 'content': message})

    print('History is')
    print(history)
    print('and messages is')
    print(messages)

    response = client.chat(
        model = 'llama3.2',
        messages = messages,
        stream = True 
    )
    
    full_response = ''
    for chunk in response:
        if 'content' in chunk['message']:
            full_response += chunk['message']['content']
            yield full_response

In [10]:
!ollama pull exaone3.5:7.8b


pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest 
pulling fffbdeec0c33... 100% ▕████████████████▏ 4.8 GB                         
pulling 37cddd3bd818... 100% ▕████████████████▏  375 B                         
pulling 8cd06db3b613... 100% ▕████████████████▏   62 B                         
pulling 294fd63925d8... 100% ▕████████████████▏  13 KB                         
pulling a64d9e642d7b... 100% ▕████████████████▏   62 B                         
pulling 339c5d906976... 100% ▕████████████████▏  563 B                         
verifying sha256 digest 
writing manifest 
success 


In [11]:
view = gr.ChatInterface(
    fn = llama_chat
)
view.launch()

/opt/anaconda3/envs/udemy_llm/lib/python3.13/site-packages/gradio/chat_interface.py:338: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


History is
[]
and messages is
[{'role': 'system', 'content': "You are helpful assistant in a clothes store. you should try to gently encourage     the customer to try items that are on sale. Hats are 60% off, and most other items are 50% off.    For example, if the customer says 'I'm looking to buy a hat',    you could reply something like, 'wonderful - we have lots of hats - incruding several that are part of our sales event.'    Encourage the customer to buy hats if they are unsure what to get."}, {'role': 'user', 'content': '안녕하세요'}]


In [12]:
gr.ChatInterface(
    fn = llama_chat
).launch()

/opt/anaconda3/envs/udemy_llm/lib/python3.13/site-packages/gradio/chat_interface.py:338: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


History is
[['안녕하세요', 'Annyeonghaseyo! Welcome to our clothes store! How can I help you today? Are you looking for something in particular or just browsing?']]
and messages is
[{'role': 'system', 'content': "You are helpful assistant in a clothes store. you should try to gently encourage     the customer to try items that are on sale. Hats are 60% off, and most other items are 50% off.    For example, if the customer says 'I'm looking to buy a hat',    you could reply something like, 'wonderful - we have lots of hats - incruding several that are part of our sales event.'    Encourage the customer to buy hats if they are unsure what to get."}, {'role': 'user', 'content': '안녕하세요'}, {'role': 'assistant', 'content': 'Annyeonghaseyo! Welcome to our clothes store! How can I help you today? Are you looking for something in particular or just browsing?'}, {'role': 'user', 'content': '한글로 대화 할 수 있을까요?'}]
History is
[['안녕하세요', 'Annyeonghaseyo! Welcome to our clothes store! How can I help you tod